In [1]:
using DataFrames, CSV, Dates, Plots, StatsPlots, MLDataPattern, MLBase, StatsPlots,Combinatorics, Optim,
DataTables, PyCall, Distributions, SpecialFunctions

In [2]:
using NBInclude
@nbinclude("Functions.ipynb")

PCA_principal_components (generic function with 1 method)

In [3]:
#Fichiers contenant les données de précipitation des pluviomètres de la ville de Montréal
data_files = readdir(raw"C:\Users\takum\Downloads\Recherche\TakumiTherville\AAA - Final(&) scripts\Données\data_all_OBXL_6_sites")[1:end]

51-element Vector{String}:
 "data_1011_daily.csv"
 "data_1021_daily.csv"
 "data_1031_daily.csv"
 "data_1041_daily.csv"
 "data_1051_daily.csv"
 "data_1061_daily.csv"
 "data_1071_daily.csv"
 "data_1081_daily.csv"
 "data_1091_daily.csv"
 "data_1101_daily.csv"
 "data_1111_daily.csv"
 "data_1121_daily.csv"
 "data_1131_daily.csv"
 ⋮
 "data_1491_daily.csv"
 "data_1501_daily.csv"
 "data_1521_daily.csv"
 "data_1531_daily.csv"
 "data_1541_daily.csv"
 "data_1551_daily.csv"
 "data_1561_daily.csv"
 "data_1571_daily.csv"
 "data_1581_daily.csv"
 "data_1591_daily.csv"
 "data_1601_daily.csv"
 "data_1611_daily.csv"

In [5]:
varNames = String.([:d1h, :d2h, :d3h, :d4h, :d6h, :d8h, :d12h, :d24h])
toPredict = "CSO"
result_folder = raw"Résultats\All OBXL\\"

"C:\\Users\\takum\\Downloads\\Recherche\\TakumiTherville\\AAA - Final(&) scripts\\Résultats\\All OBXL\\"

# Calcul du meilleur modèle (sur tous les pluviomètres et choix de variables) pour chaque ouvrage de surverse

In [3]:
#Fichiers contenant les données de surverse
site_files = readdir(raw"C:\Users\takum\Downloads\Recherche\TakumiTherville\AAA - Final(&) scripts\Données\data_all_sites_all_OBXL")[1:end]

174-element Vector{String}:
 "data_0642-01D_daily_all_OBXL.csv"
 "data_0672-01D_daily_all_OBXL.csv"
 "data_0672-02D_daily_all_OBXL.csv"
 "data_0672-03D_daily_all_OBXL.csv"
 "data_0801-01D_daily_all_OBXL.csv"
 "data_0801-02D_daily_all_OBXL.csv"
 "data_0801-03D_daily_all_OBXL.csv"
 "data_0801-04D_daily_all_OBXL.csv"
 "data_0801-05D_daily_all_OBXL.csv"
 "data_0801-06D_daily_all_OBXL.csv"
 "data_0801-07D_daily_all_OBXL.csv"
 "data_0801-08D_daily_all_OBXL.csv"
 "data_0801-09D_daily_all_OBXL.csv"
 ⋮
 "data_4620-05D_daily_all_OBXL.csv"
 "data_4620-06D_daily_all_OBXL.csv"
 "data_4620-07D_daily_all_OBXL.csv"
 "data_4620-08D_daily_all_OBXL.csv"
 "data_4630-01D_daily_all_OBXL.csv"
 "data_4710-01D_daily_all_OBXL.csv"
 "data_4710-02D_daily_all_OBXL.csv"
 "data_4720-01D_daily_all_OBXL.csv"
 "data_4760-01D_daily_all_OBXL.csv"
 "data_4770-01D_daily_all_OBXL.csv"
 "data_4790-01D_daily_all_OBXL.csv"
 "data_4795-01D_daily_all_OBXL.csv"

In [ ]:
best_models = DataFrame()

for filename in site_files
    
    site_name = filename[6:13]
    println(site_name)
    
    #if(!(site_name in best_models.Site))
    
        data = CSV.read(raw"Données\data_all_sites_all_OBXL\\" *filename, DataFrame)
        sort!(data, [:Date, :OBXL])
    
        if(!(size(data)[1] == 0))
            this_site_models = DataFrame()
    
    
            for this_pluvio in groupby(data, :OBXL)
        
                this_pluvio_models = DataFrame()
        
                OBXL_num = this_pluvio.OBXL[1]
                println(OBXL_num)
        
                for test_year in Dates.year(this_pluvio.Date[1]):Dates.year(this_pluvio.Date[end])
                    println(test_year)
    
                    train = this_pluvio[Dates.year.(this_pluvio.Date) .!= test_year, :]
                    test = this_pluvio[Dates.year.(this_pluvio.Date) .== test_year, :]
            
                    if(size(train[train.CSO .== 1, :])[1] == 0 || size(test[test.CSO .== 1, :])[1] == 0)
                        this_models = DataFrame(Variables = ["All"], F1 = [- Inf], Threshold = [1.0])
                    else
                        this_models = extensiveSearchLogistic(train, test, toPredict, varNames, 0.0)
                    end
            
                    this_models[!,:OBXL] .= OBXL_num
                    this_models[!,:Test_year] .= test_year
            
            
            
                    this_pluvio_models = vcat(this_pluvio_models, this_models)
                end
            
            
        
                this_pluvio_models = combine(groupby(this_pluvio_models, :Variables), [:F1, :Threshold] .=> mean)
                this_pluvio_models[!,:OBXL] .= OBXL_num
        
                this_site_models = vcat(this_site_models, this_pluvio_models)
        
        
        
            end
            this_site_models = DataFrame(this_site_models[argmax(this_site_models.F1_mean),:])
            this_site_models[!,:Site] .= site_name
     
            best_models = vcat(best_models, this_site_models)
        
            println(best_models)


    

            CSV.write(result_folder * " CV moyenne Occurence regression logistique seuil entrainement tout OBXL meilleur modèles.csv", best_models)
        end
    end
    
    
end

0642-01D
0672-01D
0672-02D
0672-03D
0801-01D
0801-02D
0801-03D
0801-04D
0801-05D
0801-06D
0801-07D
0801-08D
0801-09D
3230-01D
3240-01D
3240-02D
3240-03D
3240-04D
3240-05D
3250-01D
3250-02D
3260-01D
3270-01D
3275-01D
3275-02D
3280-01D
3280-02D
3290-01D
3305-01D
3305-02D
3305-03D
3305-04D
3305-05D
3305-06D
3305-07D
3310-01D
3320-01D
3330-01D
3340-01D
3350-01D
3350-02D
3350-03D
3350-04D
3350-05D
3350-06D
3350-07D
3350-08D
3350-09D
3350-10D
3350-11D
3350-12D
3370-01D
3380-01D
3390-01D
3400-01D
3410-01D
3410-02D
3411-01D
3450-01D
3450-02D
3460-01D
3480-01D
3480-02D
3480-03D
3480-04D
3480-05D
3480-06D
3490-01D
3490-02D
3500-01D
3500-02D
3500-03D
3500-04D
3520-01D
3530-01D
3540-01D
3540-02D
3560-01D
3580-01D
3580-02D
3750-01D
3762-01D
3765-01D
3767-01D
3768-01D
3780-01D
3781-01D
3782-01D
3790-01D
3790-02D
4210-01D
4230-01D
4230-02D
4230-03D
4230-04D
4230-05D
4230-06D
4230-07D
4230-08D
4230-09D
4230-10D
4240-01D
4240-02D
4250-01D
4260-01D
4265-01D
4270-01D
4270-02D
4280-01D
4280-02D
4290-01D
4

 128 │ ["d1h", "d2h", "d3h", "d6h", "d8…     0.804712       0.312165    1221  4370-05D
4380-01D
1011
2013
2014
2015
2016
2017
2018
2019
2020
1021
2013
2014
2015
2016
2017
2018
2019
2020
1031
2013
2014
2015
2016
2017
2018
2019
2020
1041
2013
2014
2015
2016
2017
2018
2019
2020
1051
2013
2014
2015
2016
2017
2018
2019
2020
1061
2013
2014
2015
2016
2017
2018
2019
2020
1071
2013
2014
2015
2016
2017
2018
2019
2020
1081
2013
2014
2015
2016
2017
2018
2019
2020
1091
2013
2014
2015
2016
2017
2018
2019
2020
1101
2013
2014
2015
2016
2017
2018
2019
2020
1111
2013
2014
2015
2016
2017
2018
2019
2020
1121
2013
2014
2015
2016
2017
2018
2019
2020
1131
2013
2014
2015
2016
2017
2018
2019
2020
1141
2013
2014
2015
2016
2017
2018
2019
2020
1151
2013
2014
2015
2016
2017
2018
2019
2020
1161
2013
2014
2015
2016
2017
2018
2019
2020
1211
2013
2014
2015
2016
2017
2018
2019
2020
1221
2013
2014
2015
2016
2017
2018
2019
2020
1231
2013
2014
2015
2016
2017
2018
2019
2020
1241
2013
2014
2015
2016
2017
2018
2019
2020
1251

 129 │ ["d1h", "d3h", "d4h", "d24h"]         0.787838       0.337487    1261  4380-01D
4390-01D
1011
2017
2018
2019
2020
1021
2017
2018
2019
2020
1031
2017
2018
2019
2020
1041
2017
2018
2019
2020
1051
2017
2018
2019
2020
1061
2017
2018
2019
2020
1071
2017
2018
2019
2020
1081
2017
2018
2019
2020
1091
2017
2018
2019
2020
1101
2017
2018
2019
2020
1111
2017
2018
2019
2020
1121
2017
2018
2019
2020
Error in model fitting
Error in model fitting
Error in model fitting
Error in model fitting
Error in model fitting
Error in model fitting
Error in model fitting
1131
2017
2018
2019
2020
1141
2017
2018
2019
2020
1151
2017
2018
2019
2020
1161
2017
2018
2019
2020
1211
2017
2018
2019
2020
1221
2017
2018
2019
2020
1231
2017
2018
2019
2020
1241
2017
2018
2019
2020
1251
2017
2018
2019
2020
1261
2017
2018
2019
2020
1271
2017
2018
2019
2020
1281
2017
2018
2019
2020
1291
2017
2018
2019
2020
1301
2017
2018
2019
2020
1311
2017
2018
2019
2020
1321
2017
2018
2019
2020
1331
2017
2018
2019
2020
1341
2017
2018
201

 130 │ ["d1h", "d6h", "d8h", "d12h", "d…     0.575          0.440983    1541  4390-01D
4400-01D
1011
2013
2014
2015
2016
2017
2018
2019
2020
1021
2013
2014
2015
2016
2017
2018
2019
2020
1031
2013
2014
2015
2016
2017
2018
2019
2020
1041
2013
2014
2015
2016
2017
2018
2019
2020
1051
2013
2014
2015
2016
2017
2018
2019
2020
1061
2013
2014
2015
2016
2017
2018
2019
2020
1071
2013
2014
2015
2016
2017
2018
2019
2020
1081
2013
2014
2015
2016
2017
2018
2019
2020
1091
2013
2014
2015
2016
2017
2018
2019
2020
1101
2013
2014
2015
2016
2017
2018
2019
2020
1111
2013
2014
2015
2016
2017
2018
2019
2020
1121
2013
2014
2015
2016
2017
2018
2019
2020
1131
2013
2014
2015
2016
2017
2018
2019
2020
1141
2013
2014
2015
2016
2017
2018
2019
2020
1151
2013
2014
2015
2016
2017
2018
2019
2020
1161
2013
2014
2015
2016
2017
2018
2019
2020
1211
2013
2014
2015
2016
2017
2018
2019
2020
1221
2013
2014
2015
2016
2017
2018
2019
2020
1231
2013
2014
2015
2016
2017
2018
2019
2020
1241
2013
2014
2015
2016
2017
2018
2019
2020
1251

4400-02D
1011
2013
2014
2015
2016
2017
2018
2019
2020
1021
2013
2014
2015
2016
2017
2018
2019
2020
1031
2013
2014
2015
2016
2017
2018
2019
2020
1041
2013
2014
2015
2016
2017
2018
2019
2020
1051
2013
2014
2015
2016
2017
2018
2019
2020
1061
2013
2014
2015
2016
2017
2018
2019
2020
1071
2013
2014
2015
2016
2017
2018
2019
2020
1081
2013
2014
2015
2016
2017
2018
2019
2020
1091
2013
2014
2015
2016
2017
2018
2019
2020
1101
2013
2014
2015
2016
2017
2018
2019
2020
1111
2013
2014
2015
2016
2017
2018
2019
2020
1121
2013
2014
2015
2016
2017
2018
2019
2020
1131
2013
2014
2015
2016
2017
2018
2019
2020
1141
2013
2014
2015
2016
2017
2018
2019
2020
1151
2013
2014
2015
2016
2017
2018
2019
2020
1161
2013
2014
2015
2016
2017
2018
2019
2020
1211
2013
Error in model fitting
2014
2015
2016
2017
2018
2019
2020
1221
2013
2014
2015
2016
2017
2018
2019
2020
1231
2013
2014
2015
2016
2017
2018
2019
2020
1241
2013
2014
2015
2016
2017
2018
2019
2020
1251
2013
2014
2015
2016
2017
2018
2019
2020
1261
2013
2014
2015
201

...

 153 │ ["d1h", "d2h", "d3h", "d8h", "d1…     0.853535       0.348299    1581  4610-01D
4610-06D
1011
2016
2017
2018
2019
2020
1021
2016
2017
2018
2019
2020
1031
2016
2017
2018
2019
2020
1041
2016
2017
2018
2019
2020
1051
2016
2017
2018
2019
2020
1061
2016
2017
2018
2019
2020
1071
2016
2017
2018
2019
2020
1081
2016
2017
2018
2019
2020
1091
2016
2017
2018
2019
2020
1101
2016
2017
2018
2019
2020
1111
2016
2017
2018
2019
2020
1121
2016
2017
2018
2019
2020
1131
2016
2017
2018
2019
2020
1141
2016
2017
2018
2019
2020
1151
2016
2017
2018
2019
2020
1161
2016
2017
2018
2019
2020
1211
2016
2017
2018
2019
2020
1221
2016
2017
2018
2019
2020
1231
2016
2017
2018
2019
2020
1241
2016
2017
2018
2019
2020
1251
2016
2017
2018
2019
2020
1261
2016
2017
2018
2019
2020
1271
2016
2017
2018
2019
2020
1281
2016
2017
2018
2019
2020
1291
2016
2017
2018
2019
2020
1301
2016
2017
2018
2019
2020
1311
2016
2017
2018
2019
2020
1321
2016
2017
2018
2019
2020
1331
2016
2017
2018
2019
2020
1341
2016
2017
2018
2019
2020
1351

 154 │ ["d2h", "d3h", "d4h", "d6h", "d8…     0.755461       0.569816    1581  4610-06D
4610-08D
1011
2016
2017
2018
2019
2020
1021
2016
2017
2018
2019
2020
1031
2016
2017
2018
2019
2020
1041
2016
2017
2018
2019
2020
1051
2016
2017
2018
2019
2020
1061
2016
2017
2018
2019
2020
1071
2016
2017
2018
2019
2020
1081
2016
2017
2018
2019
2020
1091
2016
2017
2018
2019
2020
1101
2016
2017
2018
2019
2020
1111
2016
2017
2018
2019
2020
1121
2016
2017
2018
2019
2020
1131
2016
2017
2018
2019
2020
1141
2016
2017
2018
2019
2020
1151
2016
2017
2018
2019
2020
1161
2016
2017
2018
2019
2020
1211
2016
2017
2018
2019
2020
1221
2016
2017
2018
2019
2020
1231
2016
2017
2018
2019
2020
1241
2016
2017
2018
2019
2020
1251
2016
2017
2018
2019
2020
1261
2016
2017
2018
2019
2020
1271
2016
2017
2018
2019
2020
1281
2016
2017
2018
2019
2020
1291
2016
2017
2018
2019
2020
1301
2016
2017
2018
2019
2020
1311
2016
2017
2018
2019
2020
1321
2016
2017
2018
2019
2020
1331
2016
2017
2018
2019
2020
1341
2016
2017
2018
2019
2020
1351

 155 │ ["d1h", "d2h", "d3h", "d6h", "d8…     0.78           0.241818    1401  4610-08D
4610-09D
1011
2016
2017
2018
2019
2020
1021
2016
2017
2018
2019
2020
1031
2016
2017
2018
2019
2020
1041
2016
2017
2018
2019
2020
1051
2016
2017
2018
2019
2020
1061
2016
2017
2018
2019
2020
1071
2016
2017
2018
2019
2020
1081
2016
2017
2018
2019
2020
1091
2016
2017
2018
2019
2020
1101
2016
2017
2018
2019
2020
1111
2016
2017
2018
2019
2020
1121
2016
2017
2018
2019
2020
1131
2016
2017
2018
2019
2020
1141
2016
2017
2018
2019
2020
1151
2016
2017
2018
2019
2020
1161
2016
2017
2018
2019
2020
1211
2016
2017
2018
2019
2020
1221
2016
2017
2018
2019
2020
1231
2016
2017
2018
2019
2020
1241
2016
2017
2018
2019
2020
1251
2016
2017
2018
2019
2020
1261
2016
2017
2018
2019
2020
1271
2016
2017
2018
2019
2020
1281
2016
2017
2018
2019
2020
1291
2016
2017
2018
2019
2020
1301
2016
2017
2018
2019
2020
1311
2016
2017
2018
2019
2020
1321
2016
2017
2018
2019
2020
1331
2016
2017
2018
2019
2020
1341
2016
2017
2018
2019
2020
1351

4620-01D
1011
2016
2017
2018
2019
2020
1021
2016
2017
2018
2019
2020
1031
2016
2017
2018
2019
2020
1041
2016
2017
2018
2019
2020
1051
2016
2017
2018
2019
2020
1061
2016
2017
2018
2019
2020
1071
2016
2017
2018
2019
2020
1081
2016
2017
2018
2019
2020
1091
2016
2017
2018
2019
2020
1101
2016
2017
2018
2019
2020
1111
2016
2017
2018
2019
2020
1121
2016
2017
2018
2019
2020
1131
2016
2017
2018
2019
2020
1141
2016
2017
2018
2019
2020
1151
2016
2017
2018
2019
2020
1161
2016
2017
2018
2019
2020
1211
2016
2017
2018
2019
2020
1221
2016
2017
2018
2019
2020
1231
2016
2017
2018
2019
2020
1241
2016
2017
2018
2019
2020
1251
2016
2017
2018
2019
2020
1261
2016
2017
2018
2019
2020
1271
2016
2017
2018
2019
2020
1281
2016
2017
2018
2019
2020
1291
2016
2017
2018
2019
2020
1301
2016
2017
2018
2019
2020
1311
2016
2017
2018
2019
2020
1321
2016
2017
2018
2019
2020
1331
2016
2017
2018
2019
2020
1341
2016
2017
2018
2019
2020
1351
2016
2017
2018
2019
2020
1361
2016
2017
2018
2019
2020
1401
2016
2017
2018
2019
2020
1

4620-02D
1011
2016
2017
2018
2019
2020
1021
2016
2017
2018
2019
2020
1031
2016
2017
2018
2019
2020
1041
2016
2017
2018
2019
2020
1051
2016
2017
2018
2019
2020
1061
2016
2017
2018
2019
2020
1071
2016
2017
2018
2019
2020
1081
2016
2017
2018
2019
2020
1091
2016
2017
2018
2019
2020
1101
2016
2017
2018
2019
2020
1111
2016
2017
2018
2019
2020
1121
2016
2017
2018
2019
2020
1131
2016
2017
2018
2019
2020
1141
2016
2017
2018
2019
2020
1151
2016
2017
2018
2019
2020
1161
2016
2017
2018
2019
2020
1211
2016
2017
2018
2019
2020
1221
2016
2017
2018
2019
2020
1231
2016
2017
2018
2019
2020
1241
2016
2017
2018
2019
2020
1251
2016
2017
2018
2019
2020
1261
2016
2017
2018
2019
2020
1271
2016
2017
2018
2019
2020
1281
2016
2017
2018
2019
2020
1291
2016
2017
2018
2019
2020

In [ ]:
# calcul complémentaire pour les ouvrages qui n'ont pas de pred a cause du manque de surverse sur la tranche de temps considérée

In [15]:
#best_models = DataFrame()
best_models = CSV.read(result_folder * " CV moyenne Occurence regression logistique seuil entrainement tout OBXL meilleur modèles.csv", DataFrame)

for filename in site_files
    
    site_name = filename[6:13]
    println(site_name)
    
    if !(site_name in best_models.Site)
        continue
    end
    
    i = findfirst(==(site_name), best_models.Site)
    
    if(best_models[i,:Variables] == "All")
    
        data = CSV.read(raw"Données\data_all_sites_all_OBXL\\" *filename, DataFrame)
        sort!(data, [:Date, :OBXL])
    
        if(!(size(data)[1] == 0))
            this_site_models = DataFrame()
            
            skip_all = true
    
    
            for this_pluvio in groupby(data, :OBXL)
        
                this_pluvio_models = DataFrame()
        
                OBXL_num = this_pluvio.OBXL[1]
                skip = false
        
                for test_year in Dates.year(this_pluvio.Date[1]):Dates.year(this_pluvio.Date[end])
    
                    train = this_pluvio[Dates.year.(this_pluvio.Date) .!= test_year, :]
                    test = this_pluvio[Dates.year.(this_pluvio.Date) .== test_year, :]
                    
                    if(size(train[train.CSO .== 1, :])[1] == 0 || size(test[test.CSO .== 1, :])[1] == 0)
                        #println("No cso OBXL ", OBXL_num)
                        skip = true
                    elseif(size(train[train.CSO .== 1, :])[1] == 0 || size(test[test.CSO .== 1, :])[1] == 0)
                        println("No cso test year ", test_year)
                    else
                        this_models = extensiveSearchLogistic(train, test, toPredict, varNames, 0.0)
                        this_models[!,:OBXL] .= OBXL_num
                        this_models[!,:Test_year] .= test_year
            
            
            
                        this_pluvio_models = vcat(this_pluvio_models, this_models)
                        
                        skip_all = false
                    end
            
                    
                end
                
                
                if skip == false
        
                    this_pluvio_models = combine(groupby(this_pluvio_models, :Variables), [:F1, :Threshold] .=> mean)
                    this_pluvio_models[!,:OBXL] .= OBXL_num
        
                    this_site_models = vcat(this_site_models, this_pluvio_models)
                    
                end
            end
            
            
            if skip_all == true
                println("No CSO found")
                best_models[i,:] = ["All", -Inf, 1.0, 1011, site_name]
            else
                this_site_models = DataFrame(this_site_models[argmax(this_site_models.F1_mean),:])
                this_site_models[!,:Site] .= site_name
                best_models[i,:] = this_site_models
                
                println(best_models)
            end
            


    

            CSV.write(result_folder * " CV moyenne Occurence regression logistique seuil entrainement tout OBXL meilleur modèles.csv", best_models)
        end
    end
    
    
end

0642-01D
0672-01D
No CSO found
0672-02D
0672-03D
0801-01D
No CSO found
0801-02D
0801-03D
No CSO found
0801-04D
0801-05D
0801-06D
No CSO found
0801-07D
0801-08D
0801-09D
3230-01D
3240-01D
No CSO found
3240-02D
No CSO found
3240-03D
No CSO found
3240-04D
No CSO found
3240-05D
No CSO found
3250-01D
3250-02D
3260-01D
3270-01D
3275-01D
3275-02D
No CSO found
3280-01D
3280-02D
3290-01D
3305-01D
No CSO found
3305-02D
3305-03D
3305-04D
No CSO found
3305-05D
No CSO found
3305-06D
No CSO found
3305-07D
No CSO found
3310-01D
3320-01D
3330-01D
3340-01D
3350-01D
3350-02D
3350-03D
3350-04D
No CSO found
3350-05D
3350-06D
3350-07D
3350-08D
3350-09D
3350-10D
No CSO found
3350-11D
3350-12D
3370-01D
3380-01D
3390-01D
3400-01D
3410-01D
3410-02D
3411-01D
3450-01D
3450-02D
3460-01D
3480-01D
No CSO found
3480-02D
3480-03D
3480-04D
3480-05D
3480-06D
3490-01D
3490-02D
3500-01D
No CSO found
3500-02D
3500-03D
3500-04D
3520-01D
No CSO found
3530-01D
No CSO found
3540-01D
3540-02D
3560-01D
No CSO found
3580-01D
358

In [ ]:
# calcul sans cv meilleur obxl pour chaque ouvrage

In [13]:
best_models = DataFrame()
#best_models = CSV.read(result_folder * " moyenne Occurence regression logistique seuil entrainement tout OBXL meilleur modèles.csv", DataFrame)

for filename in site_files
    
    site_name = filename[6:13]
    println(site_name)
    
    #if(!(site_name in best_models.Site))
    
        data = CSV.read(raw"Données\data_all_sites_all_OBXL\\" *filename, DataFrame)
        sort!(data, [:Date, :OBXL])
    
    
        this_site_models = DataFrame()
    
    
        for this_pluvio in groupby(data, :OBXL)
        
            this_pluvio_models = DataFrame()
        
            OBXL_num = this_pluvio.OBXL[1]
            println(OBXL_num)
        
            test_year = Dates.year(this_pluvio.Date[end])
            #println(test_year)
    
            train = this_pluvio[Dates.year.(this_pluvio.Date) .!= test_year, :]
            test = this_pluvio[Dates.year.(this_pluvio.Date) .== test_year, :]
            
            if(size(train[train.CSO .== 1, :])[1] == 0 || size(test[test.CSO .== 1, :])[1] == 0)
                this_models = DataFrame(Variables = ["All"], F1 = [- Inf], Threshold = [1.0])
            else
                this_models = extensiveSearchLogistic(train, test, toPredict, varNames, 0.0)
            end
            
            this_models[!,:OBXL] .= OBXL_num
            this_models[!,:Test_year] .= test_year
            
            
            
            this_pluvio_models = vcat(this_pluvio_models, this_models)
        
            this_pluvio_models[!,:OBXL] .= OBXL_num
        
            this_site_models = vcat(this_site_models, this_pluvio_models)
        
        
        
        end
        this_site_models = DataFrame(this_site_models[argmax(this_site_models.F1),:])
        this_site_models[!,:Site] .= site_name
     
        best_models = vcat(best_models, this_site_models)
        
        println(best_models)


    

        CSV.write(result_folder * " moyenne Occurence regression logistique seuil entrainement tout OBXL meilleur modèles.csv", best_models)
    
    #end
    
    
end

0642-01D
1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
1×6 DataFrame
 Row │ Variables                          F1       Threshold  OBXL   Test_year  Site     
     │ Any                                Float64  Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…      1.0   0.381966   1021       2020  0642-01D
0672-01D
1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020

0801-04D
1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
8×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01D

0801-09D
1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
13×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01

1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
17×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01D
   3 │ 

1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
20×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01D
   3 │ ["d2h", "d3h", "d4h", "d12h"]         0.66129    0.133588   1121       2020  0672-02D
   4 │ ["d1h", "d3h", "d6h"]                 0.666667   0.266489   1341       2020  0672-03D
   5 │ All                                -Inf          1.0        1011       2020  0801-01D
   6 │ All                                -Inf          1.0        1011       

...

1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
59×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01D
   3 │ 

3460-01D
1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
61×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01

1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
63×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01D
   3 │ 

1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
65×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01D
   3 │ 

LoadError: ArgumentError: column name :F1 not found in the data frame since it has no columns

# Calcul de la distance séparant les ouvrages de surverse et les différents pluviomètres

In [24]:
import XLSX

In [3]:
#Importation des coordonnées géographiques des pluviomètres de la ville de Montréal
xf_pluvio = CSV.read(raw"Données\pluvio_xy.csv", DataFrame)

,NAME,X,Y,DESCRIPT
,Int64,Float64,Float64,String
1,1011,2.63632e5,5.02809e6,"Usine de filtration, 2585 Paul-Guérin-Lajoie, Vaudreuil, Dorion"
2,1021,2.69753e5,5.0289e6,"Usine de filtration, 251 Montée-Sagala, Île-Perrot"
3,1051,2.86157e5,5.0272e6,"Station d'épuration, 500 chemin St-Bernard, Châteauguay"
4,1061,2.88272e5,5.04082e6,"Ateliers municipaux, 13001 boul. Cavendish, St-Laurent"
5,1071,2.84582e5,5.03356e6,"Aréna Dorval, 1450 Dawson ouest, Dorval"
6,1081,2.92137e5,5.03947e6,"Church & Dwight, 5485 Ferrier, Ville Mont-Royal"
7,1091,296693.0,5.04291e6,"École Lambert-Closse, 5840 St-Urbain, Montréal"
8,1101,2.95408e5,5.04767e6,"École Gabreil-Roy, 8699 boul. St-Michel, Montréal"
9,1111,2.94172e5,5.05037e6,"Travaux Publics, 4250 Charleroi, Montréal-Nord"


In [25]:
#Importation des coordonnées géographiques des ouvrages de surverse
xf_CSO = XLSX.readxlsx(raw"Données\ListeOuvragesSurverses_20200406.xlsx")[2]
xf_CSO = rename(select(DataFrame(xf_CSO["A2:E174"], :auto), [:x1, :x4, :x5]), ["Site", "X", "Y"])

,Site,X,Y
,Any,Any,Any
1,0642-01D,3.02761e5,5.0592e6
2,0672-01D,3.02386e5,5.06027e6
3,0672-02D,3.03131e5,5.06156e6
4,0672-03D,3.01668e5,5.05926e6
5,0801-01D,3.02655e5,5.04212e6
6,0801-02D,3.02606e5,5.04194e6
7,0801-03D,3.02664e5,5.04091e6
8,0801-04D,3.0294e5,5.04033e6
9,0801-05D,3.02366e5,5.04193e6


In [14]:
#Importation des meilleurs modèles sans validation croisée pour chaque paire pluviomètre ouvrage, avec les données aux 15 minutes et horaires
best_mods = CSV.read(raw"Résultats\All OBXL\ moyenne Occurence regression logistique seuil entrainement tout OBXL meilleur modèles.csv", DataFrame)
best_mods_15min = CSV.read(raw"Résultats\All OBXL\ moyenne Occurence regression logistique seuil entrainement tout OBXL meilleur modèles 15 min.csv", DataFrame)

,Variables,F1
,String,Float64
1,"[""d15min"", ""d45min"", ""d1h"", ""d2h"", ""d8h"", ""d12h"", ""d24h""]",1.0
2,All,-Inf
3,"[""d30min"", ""d45min"", ""d1h"", ""d3h"", ""d4h""]",0.661417
4,"[""d15min"", ""d30min"", ""d45min"", ""d1h"", ""d3h"", ""d6h"", ""d8h"", ""d12h"", ""d24h""]",0.680556
5,All,-Inf
6,All,-Inf
7,All,-Inf
8,"[""d15min"", ""d30min"", ""d45min"", ""d1h"", ""d3h"", ""d4h""]",0.909091
9,"[""d15min"", ""d30min"", ""d45min"", ""d1h"", ""d2h"", ""d4h"", ""d6h"", ""d8h"", ""d12h""]",0.686614


In [29]:
best_mods[!, :OBXL_X] = [xf_pluvio.X[xf_pluvio.NAME .== best_mods.OBXL[i]] for i in 1:size(best_mods.OBXL)[1]][:,1]
best_mods[!, :OBXL_X] .= reduce(vcat, best_mods[:,:OBXL_X])
best_mods[!, :OBXL_Y] = [xf_pluvio.Y[xf_pluvio.NAME .== best_mods.OBXL[i]] for i in 1:size(best_mods.OBXL)[1]][:,1]
best_mods[!, :OBXL_Y] .= reduce(vcat, best_mods[:,:OBXL_Y])

best_mods[!, :Site_X] = [xf_CSO.X[xf_CSO.Site .== best_mods.Site[i] ] for i in 1:size(best_mods.Site)[1]][:,1]
best_mods[!, :Site_X] .= reduce(vcat, best_mods[:,:Site_X])
best_mods[!, :Site_Y] = [xf_CSO.Y[xf_CSO.Site .== best_mods.Site[i] ] for i in 1:size(best_mods.Site)[1]][:,1]
best_mods[!, :Site_Y] .= reduce(vcat, best_mods[:,:Site_Y])

println(describe(best_mods))

best_mods[!, :Distance] = sqrt.((best_mods[:, :OBXL_X] .- best_mods[:, :Site_X]) .^2 .+ (best_mods[:, :OBXL_Y] .- best_mods[:, :Site_Y]) .^2)
sort!(best_mods, :Distance)
describe(best_mods)

10×7 DataFrame
 Row │ variable   mean       min        median     max        nmissing  eltype   
     │ Symbol     Union…     Any        Union…     Any        Int64     DataType 
─────┼───────────────────────────────────────────────────────────────────────────
   1 │ Variables             All                   ["d6h"]           0  String
   2 │ F1         -Inf       -Inf       0.740323   1.0               0  Float64
   3 │ Threshold  0.5571     0.0278302  0.381966   1.0               0  Float64
   4 │ OBXL       1209.6     1011       1121.0     1611              0  Int64
   5 │ Test_year  2019.8     2016       2020.0     2020              0  Int64
   6 │ Site                  0642-01D              4795-01D          0  String15
   7 │ OBXL_X     2.83451e5  2.63632e5  2.8571e5   3.04806e5         0  Float64
   8 │ OBXL_Y     5.03873e6  5.02809e6  5.03928e6  5.05971e6         0  Float64
   9 │ Site_X     2.92827e5  269039.0   2.94439e5  3.06365e5         0  Float64
  10 │ Site_Y     5.044

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,Variables,,All,,"[""d6h""]",0,String
2,F1,-Inf,-Inf,0.740323,1.0,0,Float64
3,Threshold,0.5571,0.0278302,0.381966,1.0,0,Float64
4,OBXL,1209.6,1011,1121.0,1611,0,Int64
5,Test_year,2019.8,2016,2020.0,2020,0,Int64
6,Site,,0642-01D,,4795-01D,0,String15
7,OBXL_X,2.83451e5,2.63632e5,2.8571e5,3.04806e5,0,Float64
8,OBXL_Y,5.03873e6,5.02809e6,5.03928e6,5.05971e6,0,Float64
9,Site_X,2.92827e5,269039.0,2.94439e5,3.06365e5,0,Float64


In [31]:
best_mods_15min[!, :OBXL_X] = [xf_pluvio.X[xf_pluvio.NAME .== best_mods_15min.OBXL[i]] for i in 1:size(best_mods_15min.OBXL)[1]][:,1]
best_mods_15min[!, :OBXL_X] .= reduce(vcat, best_mods_15min[:,:OBXL_X])
best_mods_15min[!, :OBXL_Y] = [xf_pluvio.Y[xf_pluvio.NAME .== best_mods_15min.OBXL[i]] for i in 1:size(best_mods_15min.OBXL)[1]][:,1]
best_mods_15min[!, :OBXL_Y] .= reduce(vcat, best_mods_15min[:,:OBXL_Y])

best_mods_15min[!, :Site_X] = [xf_CSO.X[xf_CSO.Site .== best_mods_15min.Site[i] ] for i in 1:size(best_mods_15min.Site)[1]][:,1]
best_mods_15min[!, :Site_X] .= reduce(vcat, best_mods_15min[:,:Site_X])
best_mods_15min[!, :Site_Y] = [xf_CSO.Y[xf_CSO.Site .== best_mods_15min.Site[i] ] for i in 1:size(best_mods_15min.Site)[1]][:,1]
best_mods_15min[!, :Site_Y] .= reduce(vcat, best_mods_15min[:,:Site_Y])

best_mods_15min[!, :Distance] = sqrt.((best_mods_15min[:, :OBXL_X] .- best_mods_15min[:, :Site_X]) .^2 .+ (best_mods_15min[:, :OBXL_Y] .- best_mods_15min[:, :Site_Y]) .^2)
sort!(best_mods_15min, :Distance)
describe(best_mods_15min)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,Variables,,All,,"[""d6h""]",0,String
2,F1,-Inf,-Inf,0.758291,1.0,0,Float64
3,Threshold,0.562073,0.0368127,0.381966,1.0,0,Float64
4,OBXL,1211.17,1011,1121.0,1601,0,Int64
5,Test_year,2019.8,2016,2020.0,2020,0,Int64
6,Site,,0642-01D,,4795-01D,0,String15
7,OBXL_X,2.83281e5,2.63632e5,2.8571e5,3.04806e5,0,Float64
8,OBXL_Y,5.03872e6,5.02809e6,5.03815e6,5.05971e6,0,Float64
9,Site_X,2.92827e5,269039.0,2.94439e5,3.06365e5,0,Float64


In [32]:
CSV.write(result_folder* "Best_models_distance.csv", best_mods)
CSV.write(result_folder* "Best_models_15min_distance.csv", best_mods_15min)

"Best_models_15min_distance.csv"